# **Conexión: Cloud y GPU**

In [ ]:
# Conectar a Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    COLAB = True
    print("Nota: Conectado a Google Drive")
except:
    print("Nota: Sin conexión a Google Drive")
    COLAB = False

# Colab Pro
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Nota: Conectado a Google Drive
Tue May 28 18:16:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default 

In [ ]:
# Clean local variables
%reset -f

# Librerías ambiente Python
import pandas as pd
import numpy as np

# Pre-processing
#from keras_preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator

# Librerías de aprendizaje profundo
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers

# Librerias para Reportes
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import normalize

# Librerías de transferencia de aprendizaje
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.applications.efficientnet import EfficientNetB0
from keras.applications.efficientnet import preprocess_input

# **Definir conjunto de datos**

## Clases y constantes globales

In [ ]:
# Leer datos
data_dir = '/content/drive/MyDrive/TG/'
image_dir = data_dir + 'IMAGES/'
filepath = data_dir + 'etiquetas444.csv'

# Importar datos con nombres de imágenes y etiquetas
data = pd.read_csv(filepath, sep=';', dtype=str)
data.body_fat = data.body_fat.astype(int)
data.id = data.id + '.png'

# Definir clases de porcentaje de grasa
Limit = [20,30]
conditions = [
    (data.body_fat < Limit[0]),
    (data.body_fat >= Limit[0]) & (data.body_fat < Limit[1]),
    (data.body_fat >= Limit[1])
    ]
Clases = ['Atlético', 'Normal', 'Sobrepeso']
data['label'] = np.select(conditions, Clases)
print(data)

# Constantes globales
seed = 73                 # Semilla para reproducir resultado
num_clases = len(Clases)
IMAGE_SIZE = [500,500]    # Píxeles
test_split = 0.20         # Porcentaje de grupo test

          id  body_fat      label
0    001.png        40  Sobrepeso
1    002.png        12   Atlético
2    004.png        10   Atlético
3    005.png        40  Sobrepeso
4    006.png        15   Atlético
..       ...       ...        ...
439  507.png        35  Sobrepeso
440  508.png        40  Sobrepeso
441  509.png        38  Sobrepeso
442  510.png        30  Sobrepeso
443  511.png        30  Sobrepeso

[444 rows x 3 columns]



## Augmentation

In [ ]:
# Aumentador de imágenes
datagen = ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest')

# Parámetros comunes entre conjuntos
datagen = ImageDataGenerator(rescale = 1./255, validation_split = test_split)

# Train set
train_set = datagen.flow_from_dataframe(
  validate_filenames=False,
  dataframe=data,
  directory=image_dir,
  x_col="id",
  y_col="label",
  subset="training",
  batch_size=32,
  seed=seed,
  shuffle=True,
  class_mode="categorical",
  target_size=IMAGE_SIZE
)

# Test set
test_set = datagen.flow_from_dataframe(
  validate_filenames=False,
  dataframe=data,
  directory=image_dir,
  x_col="id",
  y_col="label",
  subset="validation",
  batch_size=32,
  seed=seed,
  shuffle=True,
  class_mode="categorical",
  target_size=IMAGE_SIZE
  )

Found 356 non-validated image filenames belonging to 3 classes.
Found 88 non-validated image filenames belonging to 3 classes.


# **Función para *Transfer Learning***

In [ ]:
# Función para transferir aprendizaje desde red pública
def transferir_red():

  # Eliminar modelos Keras en memoria
  tf.keras.backend.clear_session()

  # Escoger red pre-entrenada
  modelo_transferencia = MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
  #modelo_transferencia = EfficientNetB0(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
  #modelo_transferencia = VGG16(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)
  #modelo_transferencia = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

  # Bloquear capas no entrenables
  for layer in modelo_transferencia.layers: layer.trainable = False

  # Modificar última capa de red pre-entrenada
  x = Flatten()(modelo_transferencia.output)
  prediction = Dense(num_clases, activation='softmax')(x)
  modelo_transferido = Model(inputs = modelo_transferencia.input, outputs = prediction)

  # Definir optimizador del modelo calsificador
  modelo_transferido.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

  return modelo_transferido

# **Entrenar modelo**

## Entrenar modelo

In [ ]:
# Transferir aprendizaje:
Modelo = transferir_red()

# Entrenar modelo
epochs = 10
history = Modelo.fit(train_set,
                     validation_data = test_set,
                     epochs = epochs)

17225924/17225924 [==============================] - 1s 0us/step
Epoch 1/10
12/12 [==============================] - 249s 20s/step - loss: 9.3500 - accuracy: 0.4719 - val_loss: 2.6126 - val_accuracy: 0.8182
Epoch 2/10
12/12 [==============================] - 15s 1s/step - loss: 2.5930 - accuracy: 0.7921 - val_loss: 4.3606 - val_accuracy: 0.7273
Epoch 3/10
12/12 [==============================] - 15s 1s/step - loss: 1.2923 - accuracy: 0.9242 - val_loss: 1.8183 - val_accuracy: 0.7955
Epoch 4/10
12/12 [==============================] - 15s 1s/step - loss: 0.3200 - accuracy: 0.9691 - val_loss: 1.4354 - val_accuracy: 0.9205
Epoch 5/10
12/12 [==============================] - 15s 1s/step - loss: 0.1189 - accuracy: 0.9916 - val_loss: 1.3854 - val_accuracy: 0.8523
Epoch 6/10
12/12 [==============================] - 15s 1s/step - loss: 0.0140 - accuracy: 1.0000 - val_loss: 1.4455 - val_accuracy: 0.8977
Epoch 7/10
12/12 [==============================] - 15s 1s/step - loss: 0.0372 - accuracy: 0.

## Prediction

In [ ]:
# Vector verdadero
true_classes = pd.DataFrame(test_set.labels)
true_classes.replace({0:Clases[0],
                      1:Clases[1],
                      2:Clases[2]},
                     inplace=True)
#true_classes

# Matrix of probabilities
y_pred = Modelo.predict(test_set)
Prediction = pd.DataFrame(y_pred, columns=Clases)

# Vectores de prediction
predicted_classes = Prediction.idxmax(axis="columns")
#print(predicted_classes)

# Matrix de confusion
matrix = confusion_matrix(true_classes, predicted_classes)
#print(matrix)

# Normalize matrix
normed_matrix = normalize(matrix, axis=1, norm='l1')
#print(normed_matrix)

confusion_df = pd.DataFrame(normed_matrix, index=Clases, columns=Clases)
confusion_df

# Mapa de calor
#import seaborn #seaborn.heatmap(confusion_df, annot=True)

3/3 [==============================] - 4s 1s/step


,Atlético,Normal,Sobrepeso
Atlético,0.531915,0.042553,0.425532
Normal,0.555556,0.111111,0.333333
Sobrepeso,0.531250,0.062500,0.406250
